In [1]:
# Define all functions needed from Task 8

def category_match_score(preferred, category):
    return 1.0 if category in preferred else 0.0

def budget_fit_score(budget_range, price):
    return 1.0 if budget_range[0] <= price <= budget_range[1] else 0.0

def rating_score(rating):
    return rating / 5.0

def compute_score(exp_row, user_vector):
    w = user_vector["weights"]
    return (
        w["rating"] * rating_score(exp_row["Rating"]) +
        w["category"] * category_match_score(user_vector["preferred_categories"], exp_row["Category"]) +
        w["budget"] * budget_fit_score(user_vector["budget_range"], exp_row["Price"])
    )

def recommend_experiences(user_vector, experiences_df, top_m=3):
    df = experiences_df.copy()
    df["City"] = df["City"].str.lower()
    df["Score"] = df.apply(lambda row: compute_score(row, user_vector), axis=1)
    top_by_city = (
        df.sort_values(by=["City", "Score"], ascending=[True, False])
          .groupby("City")
          .head(top_m)
          .reset_index(drop=True)
    )
    return top_by_city[["City", "Experience", "Category", "Price", "Rating", "Score"]]
import pandas as pd

# Step 1: Load the experience data from Task 8
experiences_df = pd.read_csv("../data/experiences_df.csv")

# Step 2: Reuse your recommend_experiences function (copy it from Task 8)

# Step 3: Simulate 3 user profiles
test_users = [
    {
        "name": "Explorer A",
        "preferred_categories": ["Cultural", "Historical"],
        "budget_range": (60, 140),
        "min_rating": 4.0,
        "weights": {"rating": 0.5, "category": 0.3, "budget": 0.2}
    },
    {
        "name": "Budget Foodie",
        "preferred_categories": ["Food"],
        "budget_range": (30, 80),
        "min_rating": 4.2,
        "weights": {"rating": 0.4, "category": 0.3, "budget": 0.3}
    },
    {
        "name": "Adventurous Nomad",
        "preferred_categories": ["Adventure", "Nature"],
        "budget_range": (40, 160),
        "min_rating": 3.8,
        "weights": {"rating": 0.3, "category": 0.5, "budget": 0.2}
    }
]

# Step 4: Define evaluation function
def evaluate_recommendations(user_vector, recommendations, top_m):
    relevant = recommendations["Category"].isin(user_vector["preferred_categories"]).sum()
    diversity = recommendations["Category"].nunique()
    coverage = 1 if len(recommendations) >= top_m else 0
    return {
        "Relevance": round(relevant / top_m, 2),
        "Diversity": diversity,
        "Coverage": coverage
    }

# Step 5: Run evaluation loop
results = []

for user in test_users:
    recs = recommend_experiences(user, experiences_df, top_m=3)
    metrics = evaluate_recommendations(user, recs, top_m=3)
    metrics["User"] = user["name"]
    results.append(metrics)

# Step 6: Show results
evaluation_df = pd.DataFrame(results)
evaluation_df = evaluation_df[["User", "Relevance", "Diversity", "Coverage"]]
evaluation_df


,User,Relevance,Diversity,Coverage
0,Explorer A,6.67,5,1
1,Budget Foodie,2.67,5,1
2,Adventurous Nomad,6.33,5,1
